In [ ]:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os 
import seaborn as sns

hakemisto = './data/'

sisalto = os.listdir(hakemisto)
print(sisalto)

# Ladataan data 
df_train = pd.read_csv('./data/Tampere_BNB_training_listing.csv')
df_test = pd.read_csv('./data/Tampere_BNB_testing_listing.csv')

In [ ]:
df_train.describe()


In [ ]:
df_train.head(100)

In [ ]:
puuttuvat_arvot = df_train.isnull().sum()
puuttuvat_arvot = puuttuvat_arvot[puuttuvat_arvot > 0]
print(f"Puuttuvat arvot:\n {puuttuvat_arvot}")

In [ ]:
puuttuvat_arvot_prosentteina = (puuttuvat_arvot / len(df_train)) * 100
print(f"Puuttuvat arvot prosentteina:\n {puuttuvat_arvot_prosentteina}")

In [ ]:
puuttuvat_arvot_prosentteina = puuttuvat_arvot_prosentteina.sort_values() # Järjestellään kasvavaan järjestykseen

# Visualisointi 
sns.set_theme(style="whitegrid")
plt.figure(figsize=(10, 15))  # Säädä kokoa tarpeen mukaan
sns.barplot(x=puuttuvat_arvot_prosentteina, y=puuttuvat_arvot_prosentteina.index)
plt.title('Puuttuvien arvojen prosenttiosuus sarakkeittain')
plt.xlabel('Prosenttiosuus (%)')
plt.ylabel('Sarakkeet')
plt.show()

In [ ]:
# Muutetaan Rv sarakkeen tyyppi kokonaisluvuksi
df_train['Rv'] = df_train['Rv'].astype(int)

In [ ]:
df_train['Rv'].head(10)

In [ ]:
df_train['Kunto'] = df_train['Kunto'].fillna('Ei tietoa')
df_train['Kunto'].value_counts().plot(kind='bar')

In [ ]:
kunto_mappaus = {
    'Ei tietoa': 0,
    'huono': 1,
    'tyyd.': 2,
    'hyvä': 3,
}
df_train['Kunto'] = df_train['Kunto'].map(kunto_mappaus)
df_train.head(10)

In [ ]:
df_train['Kunto'].value_counts()   

In [ ]:
df_train['Hissi'].value_counts()


In [ ]:
df_train['Hissi'] = df_train['Hissi'].astype('category')
df_train['Hissi'] = df_train['Hissi'].cat.codes
df_train['Hissi'].value_counts()

In [ ]:
pd.set_option('display.max_row', None) 
pd.set_option('display.max_columns', None) 
df_train['Kaupunginosa'].value_counts()

In [ ]:
df_train['Kaupunginosa'] = df_train['Kaupunginosa'].astype('category')   
df_train['Kaupunginosa'].value_counts().plot(kind='bar') 


In [ ]:
raja_arvo = 0.01
maarat = df_train['Kaupunginosa'].value_counts(normalize=True)
pienet_ryhmat = maarat[maarat < raja_arvo].index
df_train['Kaupunginosa'] = df_train['Kaupunginosa'].replace(pienet_ryhmat, 'Muu')
df_train['Kaupunginosa'].value_counts().plot(kind='bar')

In [ ]:
df_train['Kaupunginosa'].value_counts()

In [ ]:
df_train['Kaupunginosa'] = df_train['Kaupunginosa'].astype('category')
df_train['Kaupunginosa'] = df_train['Kaupunginosa'].cat.codes
df_train['Kaupunginosa'].value_counts()

In [ ]:
df_train['Asunnon tyyppi'].value_counts()

In [ ]:
tyyppimat = {
    'Yksiö' : 1,
    'Kaksi huonetta' : 2,
    'Kolme huonetta' : 3,
    'Neljä huonetta tai enemmän' : 4
}
df_train['Asunnon tyyppi'] = df_train['Asunnon tyyppi'].map(tyyppimat)
df_train['Asunnon tyyppi'].value_counts()

In [ ]:
df_train['Huoneisto'].value_counts()

In [ ]:
df_train['Huoneisto'] = df_train['Huoneisto'].str.lower()
df_train['Huoneisto'] = df_train['Huoneisto'].str.replace(' ', '')
df_train['Huoneisto'] = df_train['Huoneisto'].str.replace('+', ',')
df_train['Huoneisto'] = df_train['Huoneisto'].str.replace('...', "")
df_train['Huoneisto'].value_counts()


In [ ]:
df_train['Huoneisto'] = df_train['Huoneisto'].str.replace('/', ',')
df_train['Huoneisto'] = df_train['Huoneisto'].str.replace('[0-9]+h', '', regex=True)
df_train['Huoneisto'].value_counts()

In [ ]:
df_train['Huoneisto'] = df_train['Huoneisto'].str.replace('^,', '',regex=True)
df_train['Huoneisto'] = df_train['Huoneisto'].str.replace('^[-0-9]+', '',regex=True)
df_train['Huoneisto'] = df_train['Huoneisto'].str.replace(',$', '',regex=True)
df_train['Huoneisto'] = df_train['Huoneisto'].str.replace('^,', '',regex=True)
df_train['Huoneisto'].value_counts()

In [ ]:
huoneisto_split = df_train['Huoneisto'].str.split(',')
exploded = huoneisto_split.explode()
# exploded_unique = exploded.nunique()
exploded_unique_count = exploded.value_counts()
print(f"Unique values: {exploded_unique_count}")


In [ ]:
# Käytetään wikisivustoa https://fi.wikipedia.org/wiki/Luettelo_asuntokaupassa_k%C3%A4ytett%C3%A4vist%C3%A4_lyhenteist%C3%A4 termeille 

import re

huoneisto_split = df_train['Huoneisto'].str.split(',')

huoneisto_split = huoneisto_split.apply(lambda lst: ["parveke" if  re.search('^p$|^parv$', item) else item for item in lst])
huoneisto_split = huoneisto_split.apply(lambda lst: ["wc" if  re.search('^w$', item) else item for item in lst])
huoneisto_split = huoneisto_split.apply(lambda lst: ["sauna" if  re.search('^s$', item) else item for item in lst])
huoneisto_split = huoneisto_split.apply(lambda lst: ["2wc" if  re.search('^erill.wc$|^2xwc$', item) else item for item in lst])
huoneisto_split = huoneisto_split.apply(lambda lst: ["kph" if  re.search('^kh$', item) else item for item in lst])
huoneisto_split = huoneisto_split.apply(lambda lst: ["alkovi" if  re.search('^alk$', item) else item for item in lst])
huoneisto_split = huoneisto_split.apply(lambda lst: ["lasit" if  re.search('^l', item) else item for item in lst])
huoneisto_split = huoneisto_split.apply(lambda lst: ["avok" if  re.search('^avokeitti&#$', item) else item for item in lst])


exploded = huoneisto_split.explode()
exploded_lkm = exploded.value_counts()
print(f"Unique values: {exploded_lkm}")

In [ ]:
minimi_lkm = 10
suodatetut_idx = exploded_lkm[exploded_lkm >= minimi_lkm].index
suodatettu_lista = huoneisto_split.apply(lambda lst: [item for item in lst if item in suodatetut_idx])
suodatettu_lista = suodatettu_lista.apply(lambda lst: ["määrittämätön"] if len(lst) == 0 else lst)

df_train['Huoneisto'] = suodatettu_lista.apply(lambda lst: ','.join(lst))

# Tarkistetaan vielä vähän tymästi, että onko kaikki kunnossa
huoneisto_split = df_train['Huoneisto'].str.split(',')
exploded = huoneisto_split.explode()
exploded_lkm = exploded.value_counts()
print(f"Uniikit arvot values: {exploded_lkm}")
print(f"Määrä arvot: {exploded_lkm.count()}")


In [ ]:
df_train['Huoneisto'].head(10)

In [ ]:
print(df_train.shape)
# Splitataan 'Huoneisto' -sarake ja muunnetaan se listaksi pilkun perusteella
split_data = df_train['Huoneisto'].str.split(',')

# Käytetään explode()-metodia muuntaaksemme listan elementit omiksi riveikseen
exploded_data = split_data.explode()

# Valinnainen: Suodatetaan pois harvinaiset kategoriat ennen one-hot-enkoodausta
# value_counts = exploded_data.value_counts()
# to_keep = value_counts[value_counts >= 5].index
# filtered_data = exploded_data[exploded_data.isin(to_keep)]

# Suoritetaan one-hot-enkoodaus
one_hot_encoded = pd.get_dummies(exploded_data)

# Summataan yhteen samat rivit, koska explode() luo useita rivejä samalle alkuperäiselle indeksille
one_hot_summed = one_hot_encoded.groupby(one_hot_encoded.index).sum()

# Yhdistetään one-hot-enkoodatut sarakkeet takaisin alkuperäiseen DataFrameen
df_train = df_train.join(one_hot_summed)

print(df_train.shape)

print(df_train.head(10))


In [ ]:
df_train["Talot."].value_counts()
df_train["Talot."] = df_train["Talot."].astype('category')
df_train["Talot."] = df_train["Talot."].cat.codes
df_train["Talot."].value_counts()

In [ ]:
df_train["Krs"].value_counts()


In [ ]:
df_train["Krs"].isnull().sum()
df_train["Krs"] = df_train["Krs"].fillna("0/0")

In [ ]:
df_train["Krs"] = df_train["Krs"].str.replace('^-', '', regex=True)
df_train["Krs"].head()

In [ ]:
kerros_split = df_train["Krs"].str.split('/', expand=True)  
df_train["kerros"] = kerros_split[0].astype(int)
df_train["max_kerros"] = kerros_split[1].astype(int)    

In [ ]:
df_train.head(10)

In [ ]:
df_train.info()

In [ ]:
df_train_filtered = df_train.drop(['Huoneisto', 'Krs'], axis=1)
df_train_filtered.info()

In [ ]:
df_train_filtered.to_pickle('./data/df_train_filtered.pkl')